In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
# 통합데이터 불러오기
data = pd.read_csv('C:/taro/1차_프로젝트/통합데이터/SUWON_S_DATA_TABLE.csv',
                       encoding='utf-8')
sort_data = data.sort_values(by=['TA_YMD','HOUR'])

In [51]:
# 데이터 결측지 확인
sort_data

,TA_YMD,HOUR,AGE,DAY,AMT,CNT,DONG,UNIT,TEMP,RAIN
66627,20230101,1,2,7,17678,2,매교동,8839,2.1,0.0
66628,20230101,1,3,7,17215,9,매교동,1913,2.1,0.0
66629,20230101,1,4,7,22685,6,매교동,3781,2.1,0.0
66630,20230101,1,5,7,23689,5,매교동,4738,2.1,0.0
66631,20230101,1,6,7,20338,4,매교동,5084,2.1,0.0
...,...,...,...,...,...,...,...,...,...,...
12716,20251031,10,4,5,37704,4,화서1동,9426,12.0,0.0
12717,20251031,10,2,5,5319,3,화서2동,1773,12.0,0.0
12718,20251031,10,3,5,8562,3,화서2동,2854,12.0,0.0
12719,20251031,10,4,5,13708,3,화서2동,4569,12.0,0.0


IndexingError: Too many indexers

In [45]:
# 모형 구축
from sklearn.model_selection import train_test_split
X = np.array((sort_data['AMT']/sort_data['CNT']).round())
y = np.array(sort_data['TEMP'])
#X = mean_amt.iloc[:, :-1]
#y = temp.iloc[:, -1]
train_X, test_X, train_y, test_y = train_test_split(X, y,
                                                   test_size=0.3,
                                                   # train_size=0.7,
                                                   stratify=y # 층화추출 
                                                   )
train_X.shape, test_X.shape, train_y.shape, test_y.shape

((391639,), (167846,), (391639,), (167846,))

array([8839., 1913., 3781., ..., 2854., 4569., 1408.])

In [46]:
# 의사결정나무를 구현한 DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier(random_state=1)
dt_model.fit(train_X, train_y)

ValueError: Expected 2D array, got 1D array instead:
array=[1956. 9386. 7206. ... 7957.  986. 4014.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# =========================
# 1. 데이터 로드
# =========================
data = pd.read_csv('C:/taro/1차_프로젝트/통합데이터/SUWON_S_DATA_TABLE.csv')
df = data.sort_values(by=['TA_YMD','HOUR'])
# 날짜형 변환
df['TA_YMD'] = pd.to_datetime(df['TA_YMD'], format='%Y%m%d')

# =========================
# 2. TA_YMD 기준 일별 집계
# =========================
daily_df = (
    df
    .groupby('TA_YMD')
    .agg({
        'AMT': 'sum',
        'CNT': 'sum',
        'UNIT': 'sum',      # 🔥 타겟
        'TEMP': 'mean',
        'RAIN': 'mean',
        'AGE': 'mean',
        'DAY': 'first'      # 요일은 동일하므로 first
    })
    .reset_index()
)

# =========================
# 3. Feature / Target 분리
# =========================
X = daily_df.drop(columns=['TA_YMD', 'UNIT'])
y = daily_df['UNIT']

# =========================
# 4. 학습 / 테스트 분리
# =========================
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# =========================
# 5. 모델 학습
# =========================
model = RandomForestRegressor(
    n_estimators=300,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# =========================
# 6. 예측 및 평가
# =========================
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f"MAE  : {mae:,.2f}")
print(f"RMSE : {rmse:,.2f}")

# =========================
# 7. 결과 확인용 데이터프레임
# =========================
result_df = X_test.copy()
result_df['REAL_UNIT'] = y_test.values
result_df['PRED_UNIT'] = y_pred

result_df.sort_values(by=['])

MAE  : 289,476.00
RMSE : 435,154.23


,AMT,CNT,TEMP,RAIN,AGE,DAY,REAL_UNIT,PRED_UNIT
294,228706864,20303,12.184430,0.000000,4.958175,7,5099341,5.199796e+06
453,159309593,15037,5.370632,0.027881,5.130112,5,4817091,4.950168e+06
638,137329451,16060,20.749144,0.326560,5.174688,1,4644011,4.301125e+06
139,279489548,23934,17.294307,0.000000,5.031022,6,5488386,5.575262e+06
539,220416958,21148,23.610090,0.025899,5.125899,7,5530986,4.957370e+06
...,...,...,...,...,...,...,...,...
945,212284843,18748,24.959016,0.942397,5.178891,7,5080085,5.033948e+06
357,313126208,19908,-2.643843,0.080980,5.088235,7,6166403,5.940293e+06
485,125978577,14336,14.537655,0.000000,5.194545,2,4524994,4.332379e+06
596,134545017,15771,28.752032,0.000000,5.199647,1,4467384,4.392522e+06
